### Задание 1

Посчитайте среднее время жизни пользователей, у которых > 100 оценок (файл: ml-latest-small/ratings.csv)
* Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [71]:
movie_ratings = pd.read_csv('ml-latest-small/ratings.csv')
#movie_ratings.head()

In [48]:
count_grades = movie_ratings.groupby('userId')[['rating']].count().reset_index()
count_grades.rename(columns={'rating': 'count_grades'}, inplace=True)

In [49]:
max_date = movie_ratings.groupby('userId')[['timestamp']].max().reset_index()
max_date.rename(columns={'timestamp': 'max_date'}, inplace=True)

In [50]:
min_date = movie_ratings.groupby('userId')[['timestamp']].min().reset_index()
min_date.rename(columns={'timestamp': 'min_date'}, inplace=True)

In [66]:
full_table = count_grades[count_grades.count_grades > 100].merge(max_date, how='left',  left_on='userId', right_on='userId').merge(min_date, how='left',  left_on='userId', right_on='userId')


In [72]:
full_table['lifetime'] = full_table.max_date - full_table.min_date
full_table.head()

,userId,count_grades,max_date,min_date,lifetime
0,4,204,949982274,949778714,203560
1,8,116,1154474527,1154389340,85187
2,15,1700,1469330735,997937239,471393496
3,17,363,1127476640,1127468587,8053
4,19,423,855195373,855190091,5282


### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (файл в коде занятия)
Необходимо сформировать две таблицы:
1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2. аналогичную таблицу по типам выручки с указанием адреса клиента
*В процессе объединения таблиц данные не должны теряться

In [75]:
rzd = pd.DataFrame( # выручка железнодорожных перевозок
    { 'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981] }
)
auto = pd.DataFrame( # выручка автомобильных перевозок
    { 'client_id': [113, 114, 115, 116, 117], 
     'auto_revenue': [57483, 83, 912, 4834, 98] }
)
air = pd.DataFrame( # выручка воздушных перевозок
    { 'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173] }
)
client_base = pd.DataFrame( # адреса клиентов
    { 'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4'] }
)

In [79]:
rzd_auto = rzd.merge(auto, how='outer',  left_on='client_id', right_on='client_id')

In [82]:
rzd_auto_air = rzd_auto.merge(air, how='outer',  left_on='client_id', right_on='client_id')
rzd_auto_air

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [83]:
rzd_auto_air.merge(client_base, how='left',  left_on='client_id', right_on='client_id')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. 
Т.е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. 

Как бы вы добавили эти сведения в таблицу визитов и покупок?

*Для составления ответа можно использовать вопросы:*
1. У каждого пользователя известен набор координат. Для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на расчетах в пункте 1?
3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Связать координаты с фактом покупки:
- по времени событий получения координат и факта покупки
- по совпадению координат адреса, где сделана покупка и координат в наборе
- исключив координаты дома и работы (координаты, повторяющиеся чаще других), можно получить по совпадению повторения типа покупки и координат
- по последовательности координат установить маршруты и соотнести тип покупки и места, где их возможно было осуществить...
2. Чем больше расстояние, тем с большей вероятностью можно определить место покупки, в том числе можно учитывать время на перемещение между удаленными пунктами
3. Все зависит от того, какие еще есть данные к координатам (например, время регистрации положения) и точности оценки координат. Если есть последовательность координат, можно вычислить места длительного пребывания и по цикличности оценить дни, когда совершались те или иные передвижения, а также места интереса, а через это составить профиль личности пользователя.